# FRED-MD Dataset

**Paper:** https://doi.org/10.1080/07350015.2015.1086655      
**Homepage:** https://research.stlouisfed.org/econ/mccracken/fred-databases/    

Import the dependencies.

In [1]:
import os
import pandas as pd
import numpy as np

Define a function for transforming the time series.

In [2]:
def transform_series(x, tcode):
    '''
    Transform the time series.

    Parameters:
    ______________________________
    x: pandas.Series
        Time series.

    tcode: int.
        Transformation code.
    '''

    if tcode == 1:
        return x
    elif tcode == 2:
        return x.diff()
    elif tcode == 3:
        return x.diff().diff()
    elif tcode == 4:
        return np.log(x)
    elif tcode == 5:
        return np.log(x).diff()
    elif tcode == 6:
        return np.log(x).diff().diff()
    elif tcode == 7:
        return x.pct_change()
    else:
        raise ValueError(f"unknown `tcode` {tcode}")

Define a function for downloading and transforming the time series.

In [3]:
def get_data(year, month, transform=True):
    '''
    Download and (optionally) transform the time series.

    Parameters:
    ______________________________
    year: int
        The year of the dataset vintage.

    month: int.
        The month of the dataset vintage.

    transform: bool.
        Whether the time series should be transformed or not.
    '''

    # get the dataset URL
    file = f"https://files.stlouisfed.org/files/htdocs/fred-md/monthly/{year}-{format(month, '02d')}.csv"

    # get the time series
    data = pd.read_csv(file, skiprows=[1], index_col=0)
    data.columns = [c.upper() for c in data.columns]
    
    # process the dates
    data = data.loc[pd.notna(data.index), :]
    data.index = pd.date_range(start="1959-01-01", freq="MS", periods=len(data))

    if transform:

        # get the transformation codes
        tcodes = pd.read_csv(file, nrows=1, index_col=0)
        tcodes.columns = [c.upper() for c in tcodes.columns]

        # transform the time series
        data = data.apply(lambda x: transform_series(x, tcodes[x.name].item()))
    
    return data

Define a function for identifying the time series included in all dataset vintages between two dates.

In [4]:
def get_common_series(start_month, start_year, end_month, end_year):
    '''
    Get the list of time series included in
    all datasets vintages between two dates.

    Parameters:
    ______________________________
    start_month: int.
        The month of the start date.

    start_year: int.
        The year of the start date.

    end_month: int.
        The month of the end date.

    end_year: int.
        The year of the end date.
    '''

    # define the date range
    dates = pd.date_range(
        start=f"{start_year}-{start_month}-01",
        end=f"{end_year}-{end_month}-01",
        freq="MS"
    )

    # get the list of time series included
    # in the dataset on each date
    series = []
    for date in dates:
        series.append([c.upper() for c in
            pd.read_csv(
                f"https://files.stlouisfed.org/files/htdocs/fred-md/monthly/{date.year}-{format(date.month, '02d')}.csv",
                nrows=0,
                index_col=0
            ).columns
        ])
    
    # return the list of time series included
    # in the dataset on all dates
    return list(set.intersection(*map(set, series)))

Get the list of time series included in all dataset vintages between 01-2015 and 12-2023.

In [5]:
series = get_common_series(start_month=1, start_year=2015, end_month=12, end_year=2023)

In [6]:
len(series)

118

Load the dataset vintage for 12-2023.

In [7]:
dataset = get_data(year=2023, month=12, transform=False)

In [8]:
dataset.shape

(779, 127)

In [9]:
dataset.head()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLX,RETAILX,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTX,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSX
1959-01-01,2583.560,2426.0,15.188,276676.8154,18235.77392,21.9665,23.3891,22.2688,31.7011,19.0149,...,18.294,10.152,2.13,2.45,2.04,NaN,6476.0,12298.0,84.2043,NaN
1959-02-01,2593.596,2434.8,15.346,278713.9773,18369.56308,22.3966,23.7048,22.4617,31.9337,19.1147,...,18.302,10.167,2.14,2.46,2.05,NaN,6476.0,12298.0,83.5280,NaN
1959-03-01,2610.396,2452.7,15.491,277775.2539,18523.05762,22.7193,23.8483,22.5719,31.9337,19.4890,...,18.289,10.185,2.15,2.45,2.07,NaN,6508.0,12349.0,81.6405,NaN
1959-04-01,2627.446,2470.0,15.435,283362.7075,18534.46600,23.2032,24.1927,22.9026,32.4374,19.6138,...,18.300,10.221,2.16,2.47,2.08,NaN,6620.0,12484.0,81.8099,NaN
1959-05-01,2642.720,2486.4,15.622,285307.2201,18679.66354,23.5528,24.3936,23.1231,32.5925,20.0130,...,18.280,10.238,2.17,2.48,2.08,95.3,6753.0,12646.0,80.7315,NaN


In [10]:
dataset.tail()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLX,RETAILX,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTX,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSX
2023-07-01,19084.305,15680.1,116.235,1500218.0,694415.0,103.2166,101.2781,101.8032,102.3747,108.9528,...,118.354,123.192,29.63,34.22,26.49,71.5,498240.85,906519.21,5165.7242,13.8333
2023-08-01,19098.603,15707.7,116.171,1497717.0,699540.0,103.2158,101.4057,101.9454,102.5322,107.1502,...,120.012,123.370,29.74,34.45,26.50,69.4,504406.44,910966.86,5111.5017,15.7822
2023-09-01,19088.877,15719.1,116.594,1507530.0,705304.0,103.3374,101.1754,101.5370,102.2853,108.0366,...,120.395,123.976,29.90,34.54,26.64,67.9,508808.61,913938.95,5074.6634,15.0424
2023-10-01,19131.126,15767.9,116.682,1506491.0,703748.0,102.4189,100.4976,100.7626,101.3560,102.2019,...,120.024,124.225,29.97,34.69,26.67,63.8,511091.12,916133.37,5015.7033,19.0462
2023-11-01,19212.193,15858.9,117.056,NaN,705692.0,102.6651,100.6823,101.0283,101.4324,105.8217,...,118.977,124.535,30.20,34.96,26.87,61.3,NaN,NaN,5004.5065,13.8563


Get the mapping table with the time series description and groups.

In [11]:
def get_mapping_table():
    
    # download the zip file
    os.system("curl -O https://files.stlouisfed.org/files/htdocs/uploads/FRED-MD%20Appendix.zip")
    os.system("unzip -o FRED-MD%20Appendix.zip")
    
    # load the mapping table
    mapping = pd.read_csv(
        "FRED-MD Appendix/FRED-MD_updated_appendix.csv", 
        encoding="ISO 8859-1", 
        usecols=["fred", "description", "group"]
    )
    
    # process the time series names
    mapping["fred"] = mapping["fred"].apply(lambda x: x.upper())
    
    # process the time series descriptions  
    mapping["description"] = mapping["description"].apply(lambda x: x.encode("ascii", "ignore").decode("unicode_escape").strip())
    mapping["description"] = mapping["description"].apply(lambda x: x.replace("  ", " ").replace(" :", ":"))
    mapping["description"] = mapping["description"].apply(lambda x: x[:-1] if x.endswith(":") else x)
        
    # add the group names
    group_names = {
        1: "Output and Income",
        2: "Labor Market",
        3: "Consumption and Orders",
        4: "Orders and Inventories",
        5: "Money and Credit",
        6: "Interest Rates and Exchange Rates",
        7: "Prices",
        8: "Stock Market"
    }
    
    mapping["group_name"] = mapping["group"].apply(lambda x: group_names[x])
    
    # delete the zip file
    os.system("rm FRED-MD%20Appendix.zip")
    os.system("rm -r FRED-MD\ Appendix")
    
    return mapping.sort_values(by=["group", "description"], ignore_index=True)

<>:38: SyntaxWarning: invalid escape sequence '\ '
<>:38: SyntaxWarning: invalid escape sequence '\ '
/var/folders/m0/qtml4m217djbdp_ly0zmwpk40000gn/T/ipykernel_13807/1470411610.py:38: SyntaxWarning: invalid escape sequence '\ '
  os.system("rm -r FRED-MD\ Appendix")


In [12]:
mapping = get_mapping_table()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

Archive:  FRED-MD%20Appendix.zip
   creating: FRED-MD Appendix/
  inflating: FRED-MD Appendix/FRED-MD_historic_appendix.csv  
  inflating: FRED-MD Appendix/FRED-MD_historic_appendix.pdf  
  inflating: FRED-MD Appendix/FRED-MD_updated_appendix.csv  
  inflating: FRED-MD Appendix/FRED-MD_updated_appendix.pdf  
  inflating: FRED-MD Appendix/README.txt  


100  236k  100  236k    0     0   606k      0 --:--:-- --:--:-- --:--:--  616k


In [13]:
mapping.groupby(by="group_name")["fred"].count().sort_values(ascending=False)

group_name
Labor Market                         31
Interest Rates and Exchange Rates    22
Prices                               20
Output and Income                    16
Money and Credit                     13
Consumption and Orders               10
Orders and Inventories               10
Stock Market                          5
Name: fred, dtype: int64

In [14]:
mapping.shape

(127, 4)

In [15]:
mapping.head()

,fred,description,group,group_name
0,CUMFNS,Capacity Utilization: Manufacturing,1,Output and Income
1,INDPRO,IP Index,1,Output and Income
2,IPBUSEQ,IP: Business Equipment,1,Output and Income
3,IPCONGD,IP: Consumer Goods,1,Output and Income
4,IPDCONGD,IP: Durable Consumer Goods,1,Output and Income


In [16]:
mapping.tail()

,fred,description,group,group_name
122,S&P 500,S&Ps Common Stock Price Index: Composite,8,Stock Market
123,S&P: INDUST,S&Ps Common Stock Price Index: Industrials,8,Stock Market
124,S&P DIV YIELD,S&Ps Composite Common Stock: Dividend Yield,8,Stock Market
125,S&P PE RATIO,S&Ps Composite Common Stock: Price-Earnings Ratio,8,Stock Market
126,VIXCLSX,VIX,8,Stock Market
